In [1]:
# install package "varbvs" and load data
library(varbvs)
library(feather)

In [ ]:
# trial
data(leukemia)
# names(leukemia)
print (leukemia$y)
fit <- varbvs(leukemia$x, NULL, leukemia$y, family = "binomial", logodds = seq(-3.5,-1,0.1), sa = 1)

 [1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
[39] 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0


In [21]:
names(fit)
# Compute Monte Carlo estimates of credible intervals for coefficients in the fitted variable selection model
varbvscoefcred(fit, vars=c("X3441", "X3038", "X2141", "X2230", "X951"))

[1] "family"       "n"            "n0"           "mu.cov"       "sa0"         
 [6] "update.sigma" "update.sa"    "optimize.eta" "prior.same"   "logw"        
[11] "sigma"        "sa"           "logodds"      "alpha"        "mu"          
[16] "s"            "eta"          "model.pve"

$a
[1] -3.34594044  0.16028592 -1.17297675  0.11170526  0.08885994

$b
[1] -2.17559567  1.18617102 -0.09570761  1.21135281  1.20279401

In [2]:
# read rds file
new <- readRDS("data/del_sim_scale5_shape1_N1000_M50_999.varbvs.rds")
names(new)
dim(new$mu)

[1] "family"       "n"            "n0"           "mu.cov"       "sa0"         
 [6] "update.sigma" "update.sa"    "optimize.eta" "prior.same"   "logw"        
[11] "sigma"        "sa"           "logodds"      "alpha"        "mu"          
[16] "s"            "eta"          "model.pve"    "summary"      "w"           
[21] "pip"          "beta"

[1] 11312    26

In [1]:
# get_summary <- function(datalist){
#     for (data in datalist){
#         newdat = readRDS(data)
#         print (newdat$summary$top.vars)
#     }
# }
# get_summary(c("data/del_sim_scaleNone_shapeNone_N1000_M50_999.varbvs.rds", 
#               "data/del_sim_scaleNone_shapeNone_N1000_M100_999.varbvs.rds", 
#               "data/del_sim_scale3_shape1_N1000_M50_999.varbvs.rds", 
#               "data/del_sim_scale3_shape1_N1000_M100_999.varbvs.rds", 
#               "data/del_sim_scale5_shape1_N1000_M50_999.varbvs.rds", 
#               "data/del_sim_scale5_shape1_N1000_M100_999.varbvs.rds"))

In [ ]:
# different prior for each gene, (1-pi)*p + pi*N(0,s^2)


In [2]:
get_varbvs_rds <- function(datalist){
    for (dat in datalist){
        newdat = as.matrix(read_feather(dat))
        print (dim(newdat))
        newfit = varbvs(newdat[,-1], NULL, newdat[,1], family = "binomial", 
                        logodds = seq(-3.0,-1,0.1), sa = 1, verbose = F)
        newfit$summary = summary(newfit)
        newfit$w = normalizelogweights(newfit$logw)
        newfit$pip = newfit$alpha %*% c(newfit$w)
        newfit$beta = newfit$mu %*% c(newfit$w)
        newfit$var_names = colnames(newdat[-1])
        print (newfit$summary)
        filename = paste(dat, ".RDS", sep="")
        saveRDS(newfit, filename)
    }
}

In [3]:
get_varbvs_rds(c("data/del_sim_shape9_scale1_p0.005_N4000_M100_12.data.feather",  
                 "data/del_sim_shape9_scale1_p0.005_N4000_M200_12.data.feather", 
                 "data/del_sim_shape5_scale1_p0.005_N4000_M100_12.data.feather", 
                 "data/del_sim_shape5_scale1_p0.005_N4000_M200_12.data.feather", 
                 "data/del_sim_shape3_scale1_p0.005_N4000_M100_12.data.feather", 
                 "data/del_sim_shape3_scale1_p0.005_N4000_M200_12.data.feather", 
                 "data/del_sim_shape5_scale1_p0.01_N4000_M100_12.data.feather",
                 "data/del_sim_shape3_scale1_p0.01_N4000_M100_12.data.feather", 
                 "data/del_sim_shape3_scale1_p0.01_N4000_M200_12.data.feather", 
                 "data/del_sim_shape9_scale1_p0.01_N4000_M100_12.data.feather", 
                 "data/del_sim_shape5_scale1_p0.01_N4000_M200_12.data.feather", 
                 "data/del_sim_shape9_scale1_p0.01_N4000_M200_12.data.feather"))

[1]  4000 21756
Summary of fitted Bayesian variable selection model:
family:     binomial   num. hyperparameter settings: 21
samples:    4000       iid variable selection prior: yes
variables:  21755      fit prior var. of coefs (sa): no
covariates: 1          fit approx. factors (eta):    yes
maximum log-likelihood lower bound: -2697.3874
Hyperparameters: 
        estimate Pr>0.95             candidate values
sa            NA [NA,NA]             1--1
logodds    -1.84 [-2.00,-1.70]       (-3.00)--(-1.00)
Selected variables by probability cutoff:
>0.10 >0.25 >0.50 >0.75 >0.90 >0.95 
   36    18    15    10    10     9 
Top 5 variables by inclusion probability:
  index variable prob PVE coef*  Pr(coef.>0.95)
1   315     RBAK    1  NA  2.51 [+1.649,+3.368]
2 10668    PTPRT    1  NA  1.69 [+1.151,+2.250]
3  2308 C6orf136    1  NA  2.34 [+1.575,+3.154]
4   255 MIR219A1    1  NA  2.34 [+1.472,+3.135]
5  5267    CNNM3    1  NA  1.97 [+1.274,+2.647]
*See help(varbvs) about interpreting coeffic

In [45]:
aa <- readRDS("data/del_sim_shape3_scale1_p0.01_N4000_M100_12.data.varbvs.rds")
names(aa)
dim(aa$mu)

[1] "family"       "n"            "n0"           "mu.cov"       "sa0"         
 [6] "update.sigma" "update.sa"    "optimize.eta" "prior.same"   "logw"        
[11] "sigma"        "sa"           "logodds"      "alpha"        "mu"          
[16] "s"            "eta"          "model.pve"    "summary"      "w"           
[21] "pip"          "beta"

[1] 5046   26